In [41]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [43]:
tf.keras.backend.clear_session()

In [44]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [45]:
data = pd.read_csv('2003_mo_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Espírito Santo - Consumo de Cimento (t)
0,2003-1,8.177444,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.760059,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,98.171240,0.268445,48.426
1,2003-2,8.171622,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.760222,6.350834e+07,3.527412e+06,16.807398,5.951949e+07,97.897317,0.270512,54.969
2,2003-3,8.165799,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.760385,6.359131e+07,3.530838e+06,16.815488,5.955649e+07,98.070539,0.271941,53.847
3,2003-4,8.159977,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.760548,6.367428e+07,3.534263e+06,16.823577,5.959349e+07,98.460789,0.273321,50.375
4,2003-5,8.154154,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.760710,6.375725e+07,3.537689e+06,16.831667,5.963048e+07,99.084134,0.274453,52.576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.919726,0.505935,100.170
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.756019,0.504835,85.832
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.684567,0.502971,91.318
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.664258,0.500670,77.187


In [46]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.304933,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145
188,0.920658,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.292490,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075
189,0.915611,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.280047,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399
190,0.910564,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.267603,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917


In [47]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      48.646
1      43.597
2      56.331
3      48.488
4      52.856
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Espírito Santo - Consumo de Cimento (t), Length: 240, dtype: float64

In [48]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.538868,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.408727,1.007109,-1.505857,-1.639139,-1.626063,-0.138284,1.037783
158,1.502037,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.408716,1.017654,-1.496536,-1.616758,-1.583348,-0.199466,1.039895
159,1.465205,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.408704,1.028199,-1.487214,-1.594377,-1.540633,-0.263743,1.042387
160,1.428374,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.408692,1.038744,-1.477892,-1.571996,-1.497917,-0.329315,1.044800


In [49]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       48.646
1       43.597
2       56.331
3       48.488
4       52.856
        ...   
157     77.958
158    105.434
159     86.699
160     90.574
161     85.148
Name: Espírito Santo - Consumo de Cimento (t), Length: 162, dtype: float64

In [50]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [51]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [52]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
126,0.929165,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.503951,0.864276,-0.433841,-0.655804,0.094697,1.142222,0.875311
127,0.963455,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.514753,0.876404,-0.493112,-0.692821,0.038864,1.099243,0.884376
128,0.997745,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.525554,0.888532,-0.552384,-0.729838,-0.016968,1.049585,0.894352
129,1.032035,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.536356,0.900660,-0.611655,-0.766855,-0.072801,1.007713,0.904024
130,1.066325,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.547157,0.912788,-0.670927,-0.803872,-0.128633,0.957549,0.912127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.304933,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145
188,0.920658,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.292490,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075
189,0.915611,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.280047,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399
190,0.910564,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.267603,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917


In [53]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [54]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [55]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
#                              return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')     
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [56]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [57]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2435557205, 1971450200, 1416040889, 4294894264, 412382515, 1251049349, 2770970928, 2124946888, 1975769120, 1356335216]


Step: 0 ___________________________________________
val_loss: 85.0457763671875
winner_seed: 2435557205


Step: 1 ___________________________________________
val_loss: 92.92876434326172


Step: 2 ___________________________________________
val_loss: 110.39871215820312


Step: 3 ___________________________________________
val_loss: 91.31906127929688


Step: 4 ___________________________________________
val_loss: 87.82683563232422


Step: 5 ___________________________________________
val_loss: 115.24308013916016


Step: 6 ___________________________________________
val_loss: 96.39423370361328


Step: 7 ___________________________________________
val_loss: 85.82466888427734


Step: 8 ___________________________________________
val_loss: 91.83086395263672


Step: 9 ___________________________________________
val_loss: 92.70510864257812


final_seed: 2435557205


In [58]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 1s 75ms/step - loss: 11075.3984 - val_loss: 11168.8975
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 10976.9395 - val_loss: 11070.7539
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 10876.0225 - val_loss: 10946.1094
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 10740.4688 - val_loss: 10785.6670
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 10570.3799 - val_loss: 10595.9131
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 10374.9570 - val_loss: 10381.3438
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 10156.7256 - val_loss: 10144.9268
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 9915.4209 - val_loss: 9894.1416
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 9659.4766 - val_loss: 9625.1836
Epoch 10/10000
4/4 [====================

4/4 [==============================] - 0s 7ms/step - loss: 5216.4102 - val_loss: 5313.8511
Epoch 79/10000
4/4 [==============================] - 0s 8ms/step - loss: 5178.1895 - val_loss: 5275.1816
Epoch 80/10000
4/4 [==============================] - 0s 7ms/step - loss: 5139.9478 - val_loss: 5236.7739
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 5101.6934 - val_loss: 5198.7231
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 5064.0845 - val_loss: 5160.8818
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 5026.6885 - val_loss: 5123.2324
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 4989.5239 - val_loss: 5085.8188
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 4952.0991 - val_loss: 5048.8403
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 4915.6353 - val_loss: 5011.8530
Epoch 87/10000
4/4 [==============================] - 0s 8ms/

Epoch 155/10000
4/4 [==============================] - 0s 8ms/step - loss: 2877.3369 - val_loss: 2963.3975
Epoch 156/10000
4/4 [==============================] - 0s 8ms/step - loss: 2854.0825 - val_loss: 2940.1279
Epoch 157/10000
4/4 [==============================] - 0s 7ms/step - loss: 2830.9543 - val_loss: 2916.9768
Epoch 158/10000
4/4 [==============================] - 0s 7ms/step - loss: 2808.2886 - val_loss: 2893.8911
Epoch 159/10000
4/4 [==============================] - 0s 7ms/step - loss: 2785.5063 - val_loss: 2871.0403
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 2762.9202 - val_loss: 2848.2271
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 2740.4065 - val_loss: 2825.6555
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 2718.3318 - val_loss: 2803.1365
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 2696.0188 - val_loss: 2780.8115
Epoch 164/10000
4/4 [================

4/4 [==============================] - 0s 7ms/step - loss: 1510.6846 - val_loss: 1586.3124
Epoch 232/10000
4/4 [==============================] - 0s 7ms/step - loss: 1497.6643 - val_loss: 1572.9369
Epoch 233/10000
4/4 [==============================] - 0s 7ms/step - loss: 1484.4253 - val_loss: 1559.6022
Epoch 234/10000
4/4 [==============================] - 0s 8ms/step - loss: 1471.5692 - val_loss: 1546.2378
Epoch 235/10000
4/4 [==============================] - 0s 8ms/step - loss: 1458.3364 - val_loss: 1533.0459
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 1445.4922 - val_loss: 1519.8798
Epoch 237/10000
4/4 [==============================] - 0s 7ms/step - loss: 1432.4424 - val_loss: 1506.8699
Epoch 238/10000
4/4 [==============================] - 0s 7ms/step - loss: 1419.6744 - val_loss: 1493.9194
Epoch 239/10000
4/4 [==============================] - 0s 7ms/step - loss: 1406.8773 - val_loss: 1481.0604
Epoch 240/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 750.1285 - val_loss: 815.5182
Epoch 309/10000
4/4 [==============================] - 0s 7ms/step - loss: 743.3638 - val_loss: 808.6172
Epoch 310/10000
4/4 [==============================] - 0s 7ms/step - loss: 736.4615 - val_loss: 801.8279
Epoch 311/10000
4/4 [==============================] - 0s 7ms/step - loss: 729.8734 - val_loss: 795.0159
Epoch 312/10000
4/4 [==============================] - 0s 7ms/step - loss: 723.2500 - val_loss: 788.2340
Epoch 313/10000
4/4 [==============================] - 0s 7ms/step - loss: 716.5122 - val_loss: 781.5336
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 710.0204 - val_loss: 774.7967
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 703.6419 - val_loss: 768.0915
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 697.0879 - val_loss: 761.5809
Epoch 317/10000
4/4 [==============================] - 0s 8ms/step - lo

...

Epoch 1094/10000
4/4 [==============================] - 0s 8ms/step - loss: 34.6193 - val_loss: 89.3740
Epoch 1095/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.7139 - val_loss: 89.7488
Epoch 1096/10000
4/4 [==============================] - 0s 8ms/step - loss: 34.4098 - val_loss: 90.8362
Epoch 1097/10000
4/4 [==============================] - 0s 8ms/step - loss: 34.1517 - val_loss: 89.6411
Epoch 1098/10000
4/4 [==============================] - 0s 7ms/step - loss: 34.0700 - val_loss: 88.1529
Epoch 1099/10000
4/4 [==============================] - 0s 8ms/step - loss: 33.6109 - val_loss: 89.5864
Epoch 1100/10000
4/4 [==============================] - 0s 8ms/step - loss: 33.8013 - val_loss: 89.9430
Epoch 1101/10000
4/4 [==============================] - 0s 7ms/step - loss: 33.9470 - val_loss: 88.9857
Epoch 1102/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.5663 - val_loss: 88.9910
Epoch 1103/10000
4/4 [==============================] - 0s 8ms/s

4/4 [==============================] - 0s 7ms/step - loss: 27.3726 - val_loss: 95.7279
Epoch 1173/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.4871 - val_loss: 95.3738
Epoch 1174/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.2705 - val_loss: 94.2363
Epoch 1175/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.5930 - val_loss: 94.1913
Epoch 1176/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.4044 - val_loss: 94.6626
Epoch 1177/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.4826 - val_loss: 95.2852
Epoch 1178/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.3254 - val_loss: 96.1454
Epoch 1179/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.6636 - val_loss: 95.9006
Epoch 1180/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.2547 - val_loss: 95.6220
Epoch 1181/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.38

Epoch 1251/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.6110 - val_loss: 96.0195
Epoch 1252/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.4389 - val_loss: 95.3273
Epoch 1253/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.9363 - val_loss: 94.6039
Epoch 1254/10000
4/4 [==============================] - 0s 7ms/step - loss: 26.4848 - val_loss: 95.0350
Epoch 1255/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.5724 - val_loss: 95.8618
Epoch 1256/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.5912 - val_loss: 96.3436
Epoch 1257/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.7503 - val_loss: 96.8701
Epoch 1258/10000
4/4 [==============================] - 0s 8ms/step - loss: 26.5830 - val_loss: 94.3695
Epoch 1259/10000
4/4 [==============================] - 0s 7ms/step - loss: 33.3624 - val_loss: 93.1062
Epoch 1260/10000
4/4 [==============================] - 0s 8ms/s

4/4 [==============================] - 0s 7ms/step - loss: 29.3811 - val_loss: 94.8337
Epoch 1330/10000
4/4 [==============================] - 0s 8ms/step - loss: 29.4061 - val_loss: 97.7788
Epoch 1331/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.5930 - val_loss: 102.0144
Epoch 1332/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.7285 - val_loss: 100.9816
Epoch 1333/10000
4/4 [==============================] - 0s 8ms/step - loss: 30.0444 - val_loss: 102.2763
Epoch 1334/10000
4/4 [==============================] - 0s 7ms/step - loss: 29.0311 - val_loss: 100.9169
Epoch 1335/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.9743 - val_loss: 102.1192
Epoch 1336/10000
4/4 [==============================] - 0s 9ms/step - loss: 29.5756 - val_loss: 102.4066
Epoch 1337/10000
4/4 [==============================] - 0s 7ms/step - loss: 29.8766 - val_loss: 101.9656
Epoch 1338/10000
4/4 [==============================] - 0s 7ms/step - loss

4/4 [==============================] - 0s 8ms/step - loss: 27.2095 - val_loss: 120.8490
Epoch 1408/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.4577 - val_loss: 129.3686
Epoch 1409/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.6072 - val_loss: 134.3182
Epoch 1410/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.9059 - val_loss: 142.7322
Epoch 1411/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.6351 - val_loss: 164.1433
Epoch 1412/10000
4/4 [==============================] - 0s 8ms/step - loss: 26.8553 - val_loss: 191.1595
Epoch 1413/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.6004 - val_loss: 234.4398
Epoch 1414/10000
4/4 [==============================] - 0s 8ms/step - loss: 27.9231 - val_loss: 230.9294
Epoch 1415/10000
4/4 [==============================] - 0s 7ms/step - loss: 28.1392 - val_loss: 230.7903
Epoch 1416/10000
4/4 [==============================] - 0s 7ms/step - lo

In [59]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [60]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,88.744743,88.474571,88.032295,87.793243,87.543564,87.233788,87.242332,87.811813,88.006683,88.032959,88.206535,88.772774,88.167885,88.925087,89.760078,89.748962,89.324837,88.585548,89.383705,90.418633,90.784538,91.295296,91.615471,91.819374,92.006058,92.191032,92.894615,94.276237,95.519638,96.178642
Target,86.678,100.174,89.777,94.81,93.446,87.54,96.706,74.086,92.676,82.865,71.18,108.826,105.694,107.106,101.893,110.551,100.587,94.22,111.377,98.614,94.022,91.546,95.674,88.927,105.688,104.711,97.53,99.651,76.71,75.456
Error,2.066742,11.699432,1.744705,7.016754,5.902435,0.306213,9.463669,13.725815,4.669319,5.167961,17.026535,20.053223,17.526115,18.180916,12.132919,20.80204,11.262161,5.634453,21.993294,8.195366,3.237465,0.250702,4.058533,2.892372,13.681946,12.519966,4.635384,5.374763,18.809639,20.722641


In [61]:
display(mae)
display(mape)

10.025116

0.10658923

In [62]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [63]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1055.8954]] - Target[1078.7640000000001]| =  Error: [[22.868652]]; MAPE:[[0.02119894]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1079.8295]] - Target[1200.211]| =  Error: [[120.38159]]; MAPE:[[0.10030035]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[563.0662]] - Target[559.746]| =  Error: [[3.3202515]]; MAPE:[[0.00593171]]


[array([[22.868652]], dtype=float32),
 array([[120.38159]], dtype=float32),
 array([[3.3202515]], dtype=float32)]

48.85683

0.042476997